# 模块并行化

模块的并行化可以缩减模块的运算时间。

但是必须先确定影响算法效率的原因是算法本身还是串行的影响。

科学任务（例如数值建模）的步骤通常如下所示：   

    +------------------+                        +---------------+
    |    输入数据       |                        |               |
    |    初始状态       +----------------------->| final results |
    |                  |     长时间的计算任务     |               |
    +------------------+                        +---------------+

In GIS, we are commonly dealing with rather simple chained computations that are applied to (massive) input data requiring several heavy partial results before to obtain the result:

在GIS中，我们通常处理相当简单的链式计算，该计算适用于有大量输入数据时，需要一些重复的计算过程才能得到结果：

    +------------+
    |            |       +---------+    +---------+    +---------+
    | input data +------>| partial +--->| partial +--->| partial +
    |            |       | results |    | results |    | results |
    +------------+       +---------+    +---------+    +----+----+
                                                            |
                                                            |
    +---------------+    +---------+    +---------+    +---------+
    |               |<---+ partial |<---+ partial |<---+ partial |
    | final results |    | results |    | results |    | results |
    |               |    +---------+    +---------+    +----+----+
    +---------------+                                        

对于并行化任务，在GIS处理中，主要瓶颈通常是在数据密集型计算步骤中的最大硬盘读写速度。


在开始使用CPU的所有内核之前，请确保检查系统的读/写功能是否已饱和，例如可以使用系统监控器工具完成。

## ParallelModuleQueue 并行计算类

并行执行几个模块的简单方法（由SörenGebbert开发）是使用ParallelModuleQueue类，其基本思想是创建一个队列，其中包含执行并行计算的所有模块。

ParallelModuleQueue类基于pygrass库的Module类，下面是一个用于视域计算的小示例：

In [1]:
# import the necessary libraries
from copy import deepcopy

from grass.pygrass.modules import Module, ParallelModuleQueue
from grass.pygrass.vector import VectorTopo

In [2]:
# define the global variables with the inputs
TMP_VIEWSHED = 'tmp_viewshed_{:03}'
ELEV = 'elevation'
POINTS = 'view_points'

In [3]:
# we set the region to elevation map
region = Module('g.region', raster='elevation')

In [4]:
# initialize an empty queue and list 实例化一个空队列
queue = ParallelModuleQueue(nprocs=4)
viewsheds = []

In [5]:
# initialize a module instance with shared inputs
viewshed = Module('r.viewshed', input=ELEV, observer_elevation=3,
                  run_=False, overwrite=True)

# open the input vector map and start cycling over the vector points
with VectorTopo(POINTS, mode='r') as points:
    for point in points:
        # create a copy of the module and set the remaining parameters
        print(point.id)
        m = deepcopy(viewshed)(output=TMP_VIEWSHED.format(point.id),
                               coordinates=point.coords())
        viewsheds.append(m)
        queue.put(m)#将每次的计算放到并行队列中
    queue.wait()


1
2


In [6]:
viewsheds

[Module('r.viewshed'), Module('r.viewshed')]